# Module 5 - Programming Assignment

## Directions

There are general instructions on Blackboard and in the Syllabus for Programming Assignments. This Notebook also has instructions specific to this assignment. Read all the instructions carefully and make sure you understand them. Please ask questions on the discussion boards or email me at `EN605.445@gmail.com` if you do not understand something.

<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
You must follow the directions *exactly* or you will get a 0 on the assignment.
</div>

You must submit a zip file of your assignment and associated files (if there are any) to Blackboard. The zip file will be named after you JHED ID: `<jhed_id>.zip`. It will not include any other information. Inside this zip file should be the following directory structure:

```
<jhed_id>
    |
    +--module-05-programming.ipynb
    +--module-05-programming.html
    +--(any other files)
```

For example, do not name  your directory `programming_assignment_01` and do not name your directory `smith122_pr1` or any else. It must be only your JHED ID.

In [1]:
from IPython.core.display import *
from StringIO import StringIO
from random import random, gauss, randrange
import bisect, sys

## Local Search - Genetic Algorithm

For this assignment we're going to use the Genetic Algorithm to find the solution to a shifted Sphere Function in 10 dimensions, $x$, where the range of $x_i$ in each dimension is (-5.12 to 5.12). Here a "solution" means the vector $x$ that minimizes the function. The Sphere Function is:

$$f(x)=\sum x^2_i$$

We are going to shift it over 0.5 in every dimension:

$$f(x) = \sum (x_i - 0.5)^2$$

where $n = 10$.

As this *is* a minimization problem you'll need to use the trick described in the lecture to turn the shifted Sphere Function into an appropriate fitness function (which is always looking for a *maximum* value).

## Binary GA

You are going to solve the problem two different ways. First, using the traditional (or "Canonical") Genetic Algorithm that encodes numeric values as binary strings (you don't have to represent them literally as strings but they are general lists or strings of only 0 or 1).

There are many different ways to affect this encoding. For this assignment, the easiest is probably to use a 10 bit binary encoding for each $x_i$. This gives each $x_i$ a potential value of 0 to 1024 which can be mapped to (-5.12, 5.12) by subtracting 512 and dividing by 100.

All the GA operators will be as described in the lecture.

**Important**

Please remember that there is a difference between the *genotype* and the *phenotype*. The GA operates on the *genotype* (the encoding) and does not respect the boundaries of the phenotype (the decoding). So, for example, do **not** use a List of Lists to represent an individual. It should be a *single* List of 10 x 10 or 100 bits. In general, crossover and mutation have no idea what those bits represent.

## Real Valued GA

For the real valued GA, you can represent each $x_i$ as a float in the range (-5.12, 5.12) but you will need to create a new mutation operator that applies gaussian noise. Python's random number generator for the normal distribution is called `gauss` and is found in the random module:

```
from random import gauss, random
```

You may need to experiment a bit with the standard deviation of the noise but the mean will be 0.0.

## GA

The Genetic Algorithm itself will have the same basic structure in each case: create a population, evaluate it, select parents, apply crossover and mutation, repeat until the number of desired generations have been generated. The easiest way to accomplish this in "Functional" Python would be to use Higher Order Functions.



Your code should print out the best individual of each generation including the generation number, genotype (the representation), phenotype (the actual value), the fitness (based on your fitness function transformation) and the function value (for the shifted sphere) if passed a DEBUG=True flag.

The GA has a lot of parameters: mutation rate, crossover rate, population size, dimensions (given for this problem), number of generations.  You can put all of those and your fitness function in a `Dict` in which case you need to implement:

```python
def binary_ga( parameters):
  pass
```

and

```python
def real_ga( parameters):
  pass
```

Remember that you need to transform the sphere function into a legit fitness function. Since you also need the sphere function, I would suggest that your parameters Dict includes something like:

```python
parameters = {
   "f": lambda xs: sphere( 0.5, xs),
   "minimization": True
   # put other parameters in here.
}
```

and then have your code check for "minimization" and create an entry for "fitness" that is appropriate.

In [2]:
def sphere( shift, xs):
    return sum( [(x - shift)**2 for x in xs])

In [3]:
sphere( 0.5, [1.0, 2.0, -3.4, 5.0, -1.2, 3.23, 2.87, -4.23, 3.82, -4.61])

113.42720000000001


-----

### Common Helper Functions ###

** cumsum(L) **  
Calculate cumulative sum of a list. Given a list `x`, returns a list `y` where the n-th element of y is 
$$y_{n} = \sum_{i=1}^{n} x_{i}$$

** fitFn(pm, genes) **  
Function to calculate fitness of a genotype. Requires the parameter used for the genetic algorithm, which should contain a maxVal, a phenotype function, and the sphere function. The fitness function used is $$Fit = maxVal - Sphere(phenoFn(genotype))$$ where maxVal is the largest possible value that the sphere function could take on, the `phenoFn` is a function that transforms genotype into the actual phenotype of real values, and Sphere function is defined above. Using this fitness function ensures that the real vectors which results in the lowest shifted sphere function would end up mapped to a higher fitness value.

** pickParents(fitness, nParents) **  
Given a list of fitness values for the current population, picks a number of eligible parents as specified in the `nParents` parameter.

The function excludes all individuals which have less than average fitness from being eligible to be a parent. For those individuals deemed eligible, the program calculates an array containing the cumulative sum of the fitness values of all eligible individuals. The algorithm then generates a specified number of random values between 0 and sum(fitness). The individual selected would be the index of the left most element which is higher than the random value. The higher the fitness of an individual, the higher the probability of being selected.

To ensure the viability of the corp of parents in later iterations, when there are less than 5 eligible individuals based on average fitness, all individuals are deemed eligible.

** crossover(parents1, parents2) **  
Given two parent genotypes, generates a uniform random integer between 0 and len(genotype)-1 to be used as the index, such that the child contains the genes of the first parent up until the index concatenated with the genes of the second parent from the index to the end.

** updateBest(pop, fitness, bestGene, bestFit) **  
Given the genotypes and values of the current population, the best ever genotype and fitness value encountered so far, the function calculates the best individual genotype of the current generation, and if it is better than the all time best, will update the all time best accordingly.

** printGenInfo(genNum, gene, fit, pm) **
The function prints debug information about the progress of the genetic algorithm. The parameters are: generation number, the genotype and fitness value of the best individual of the current generation and the parameter of the algorithm. Within the parameter, the function uses the `reprFn` which nicely returns the string representation of the genotype, the phenoFn which converts the genotype to the phenotype, and the `f` evaluative function. The function printes the generation number, the representation of the genetype and the phenotype of the best individual of the generation, and the fitness and sphere values.

In [4]:
def cumsum(L):
    total = 0
    for l in L:
        total += l
        yield total

def fitFn(pm, genes):
    return pm['maxVal'] - pm['f']( pm['phenoFn'](genes) )

def pickParents(fitness, nParents):
    avgFit = sum(fitness) / len(fitness) # avg fitness
    tmp = [(n,f) for n,f in enumerate(fitness) if f>avgFit]
    if len(tmp) > 5:
        idx,eligible = zip(*tmp)
    else:
        idx,eligible = range(len(fitness)),fitness
    cumFit = list(cumsum( eligible ))
    rands = [random()*cumFit[-1] for n in xrange(nParents)]
    return [idx[bisect.bisect_left(cumFit,r)] for r in rands]

def crossover(parents1, parents2):
    pt = randrange(len(parents1))
    return parents1[:pt] + parents2[pt:]

def updateBest(pop, fitness, bestGene, bestFit):
    iterBestIdx = fitness.index(max(fitness))
    if fitness[iterBestIdx] > bestFit:
        bestGene = pop[iterBestIdx]
        bestFit = fitness[iterBestIdx]
    return bestGene, bestFit, pop[iterBestIdx], fitness[iterBestIdx]

def printGenInfo(genNum, gene, fit, pm):
    out = 'Gen %s best gene: %s\n' + \
        '\tx-values: %s \n\tfitness: %.4f, sphere value: %.4f\n'
    pheno = pm['reprFn'](gene)
    xVal = pm['phenoFn'](gene)
    xRepr = '[' + ', '.join('%.3g' % f for f in xVal) + ']'
    sp = pm['f'](xVal)
    print out % (str(genNum), str(pheno), str(xRepr), fit, sp)

### Genetic Algorithm  Main Program ###  

The Genetic Algorithm program is a general program which relies on the input parameter for various settings as well as functions. 

The program first initializes a specific number of individuals as specified in the parameter (`popSize`) by generating them randomly. Then the program iterates a specific number of times (`nGeneration`) according to the parameter, recording the best individual and its fitness value, and replacing it as needed if a generation produces a better one. Within each generation, the program picks parents by some eligibility criteria as specified in `pickParents(...)` function. For each generation, the algorithm picks twice the amount of parents as the population size since every two parents produce one offspring. For each child produced, the program applies a small probability of mutation as specified in `mutateProb`. The new generation then replaces the previous one and the algorithm repeats.

At the end, the phenotype of the best individual is returned.

In [5]:
def geneticAlgo(pm, DEBUG=False):
    pop = pm['initFn'](pm)
    
    bestGene,bestFit = None,-sys.maxsize # initialize best 
    for gen in xrange(pm['nGeneration']):
        fitness = [fitFn(pm, genes) for genes in pop]
        bestGene,bestFit,g,f = updateBest(pop, fitness, bestGene, bestFit)
        
        if DEBUG and gen%pm['printEvery']==0:
            printGenInfo(gen, g, f, pm)
        idx = pickParents(fitness, pm['popSize']*2)
        nextGen = list()
        for n in xrange(pm['popSize']):
            child = crossover(pop[idx[n]], pop[idx[-n]])
            if random() < pm['mutateProb']:
                pm['mutateFn'](child, pm)
            nextGen.append(child)
        pop = nextGen
    
    bestGene,bestFit,g,f = updateBest(pop, fitness, bestGene, bestFit)
    printGenInfo('final', bestGene, bestFit, pm)
    return pm['phenoFn'](bestGene)

-----

## Binary GA ##

### Binary GA Helper Functions###
** getDecimals(genes) **  
Given a genotype of the binary GA (binary strings), it breaks it up into 10 pieces and converts them to real values. This is use as the `phenoFn` for binary GA.

** mutateBinary(genes, pm) **  
Mutate one genotype of the binary GA algorithm by randomly selecting one binary bit within the genotype and flipping the bit. This is use as the `mutateFn` for binary GA.

** initBinGA(pm) **  
Initialize the population of the binary GA process by randomly generating all bits. This is use as the `initFn` for binary GA.

In [6]:
def getDecimals(genes):
    bins = [genes[n:n+10] for n in xrange(0,len(genes),10)]
    return [(int(x,2)-511)/100.0 for x in bins]

def mutateBinary(genes, pm):
    i = randrange(len(genes))
    genes = genes[:i] + ('0' if genes[i]=='1' else '1') + genes[i+1:]
    return genes

def initBinGA(pm):
    tmp = [[format(randrange(2**pm['nBits']-1), '#012b')[2:] \
            for x in xrange(10)] for y in xrange(pm['popSize'])]
    return [''.join(p) for p in tmp]

### Binary GA Parameters and Run ###

In [7]:
## Traditional GA

param = {
    "f": lambda xs: sphere( 0.5, xs),
    "popSize": 1000,
    "mutateProb": 0.05,
    "nGeneration": 100,
    "nBits": 10,
    "maxVal": sphere(0.5, [-5.12 for x in xrange(10)]),
    "phenoFn" : lambda x: getDecimals(x),
    "initFn": initBinGA, 
    "mutateFn": mutateBinary,
    "reprFn": lambda x: [x[n:n+10] for n in xrange(0,len(x),10)],
    "printEvery": 10
}

bestX =  geneticAlgo( param, True)

Gen 0 best gene: ['0111110101', '0110100011', '1001101010', '1011110011', '0111100111', '0110110000', '1000001010', '1011100011', '0110100010', '1011110110']
	x-values: [-0.1, -0.92, 1.07, 2.44, -0.24, -0.79, 0.11, 2.28, -0.93, 2.47] 
	fitness: 297.9211, sphere value: 17.9229

Gen 10 best gene: ['1001011101', '1000100000', '1000100001', '1000011110', '1001010110', '1000111111', '1001101101', '1001000010', '1000111100', '1000101100']
	x-values: [0.94, 0.33, 0.34, 0.31, 0.87, 0.64, 1.1, 0.67, 0.61, 0.45] 
	fitness: 314.9998, sphere value: 0.8442

Gen 20 best gene: ['1001000001', '1000100000', '1000110100', '1000111110', '1000111111', '1000110100', '1000110111', '1000101001', '1001000000', '1000001100']
	x-values: [0.66, 0.33, 0.53, 0.63, 0.64, 0.53, 0.56, 0.42, 0.65, 0.13] 
	fitness: 315.5818, sphere value: 0.2622

Gen 30 best gene: ['1001000000', '1000111100', '1000110010', '1000111011', '1000101110', '1000110000', '1000111011', '1000100000', '1000110001', '1000101110']
	x-values: [0.65

---

## Real Value GA ##

### Real Value GA Helper Functions ###

** mutateReal(genes, pm) **  
Mutate one genotype of the binary GA algorithm by randomly selecting one real value within the genotype and applying gaussian noise to it. Uses the `sigma` and `xMax` of the parameter, which is the standard deviation of the gaussian noise. The function also ensures that the post-mutation genotype stays within the confines of the x-value as specified in xMax. This is use as the `mutateFn` for binary GA.

** initRealGA(pm) **  
Initialize the population of the binary GA process by randomly generating real numbers as specified by `xMax`. This is use as the `initFn` for binary GA.

In [8]:
def mutateReal(genes, p):
    i = randrange(len(genes))
    genes[i] = min( max(genes[i]*gauss(0,p['sigma']),-p['xMax']) , p['xMax'])
    return genes

def initRealGA(pm):
    return [[random()*2*pm['xMax'] - pm['xMax'] for x in xrange(10)] \
           for y in xrange(pm['popSize'])]

### Real Value GA Parameters and Run ###

In [9]:
## Real Valued GA

param2 = {
    "f": lambda xs: sphere( 0.5, xs),
    "popSize": 1000,
    "mutateProb": 0.01,
    "nGeneration": 100,
    "nBits": 10,
    "maxVal": sphere(0.5, [-5.12 for x in xrange(10)]),
    "sigma": 0.5,
    "xMax": 5.12,
    
    "phenoFn" : lambda x: x,
    "initFn": initRealGA, 
    "mutateFn": mutateReal,
    "reprFn": lambda x: '[' + ', '.join('%.3f' % f for f in x) + ']',
    "printEvery": 10
    
}

bestXreal = geneticAlgo( param2, True)

Gen 0 best gene: [2.780, 1.534, 1.539, -0.352, 2.042, -0.820, -0.717, -1.375, -0.703, 0.855]
	x-values: [2.78, 1.53, 1.54, -0.352, 2.04, -0.82, -0.717, -1.37, -0.703, 0.855] 
	fitness: 297.0780, sphere value: 18.7660

Gen 10 best gene: [0.044, 1.144, 1.480, 0.304, 0.159, 1.017, 0.971, 0.529, 0.274, 0.445]
	x-values: [0.0436, 1.14, 1.48, 0.304, 0.159, 1.02, 0.971, 0.529, 0.274, 0.445] 
	fitness: 313.5620, sphere value: 2.2820

Gen 20 best gene: [0.470, 0.701, 0.424, 0.304, 0.261, 0.492, 0.580, 0.529, 0.420, 0.518]
	x-values: [0.47, 0.701, 0.424, 0.304, 0.261, 0.492, 0.58, 0.529, 0.42, 0.518] 
	fitness: 315.6876, sphere value: 0.1564

Gen 30 best gene: [0.470, 0.611, 0.424, 0.537, 0.421, 0.492, 0.580, 0.529, 0.420, 0.445]
	x-values: [0.47, 0.611, 0.424, 0.537, 0.421, 0.492, 0.58, 0.529, 0.42, 0.445] 
	fitness: 315.8007, sphere value: 0.0433

Gen 40 best gene: [0.470, 0.611, 0.424, 0.537, 0.421, 0.492, 0.580, 0.529, 0.420, 0.518]
	x-values: [0.47, 0.611, 0.424, 0.537, 0.421, 0.492, 0.58, 